In [9]:
import pandas as pd 

file_path = 'C:\\Users\\vardh\\.vscode\\Identifying-Fake-Job-Listing\\job_train.csv'
job_data = pd.read_csv(file_path)

job_data.head(), job_data.info(), job_data.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8940 entries, 0 to 8939
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             8940 non-null   object
 1   location          8783 non-null   object
 2   description       8939 non-null   object
 3   requirements      7614 non-null   object
 4   telecommuting     8940 non-null   int64 
 5   has_company_logo  8940 non-null   int64 
 6   has_questions     8940 non-null   int64 
 7   fraudulent        8940 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 558.9+ KB


(                                               title                location  \
 0               Architect (Middleware - MQ) - Kuwait                KW, KU,    
 1  Interviewing Now for Sales Rep Positions -- wi...  US, TX, Corpus Christi   
 2  Process Controls Staff Engineer - Foxboro I/A ...   US, TX, USA Southwest   
 3  Experienced Telemarketer Wanted - Digital Solu...               AU, NSW,    
 4                            Senior Network Engineer         GB, ENG, London   
 
                                          description  \
 0  On behalf of our client, a well known multinat...   
 1  We are Argenta Field Solutions, a rapidly expa...   
 2  Experienced Process Controls Staff Engineer is...   
 3  If you have a passion for people and love to s...   
 4  As the successful Senior Network Engineer you ...   
 
                                         requirements  telecommuting  \
 0  -Working technical knowledge of IT systems and...              0   
 1                      

In [10]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Handle missing values for 'description' and 'requirements'
job_data['description'].fillna('No information', inplace= True)
job_data['requirements'].fillna('No information', inplace= True)

# Drop rows with missing 'location' for simplicity
job_data.dropna(subset=['location'], inplace= True)

# Define feature columns and target
text_features = ['title', 'description', 'requirements']
categorial_features = ['location']
binary_features = ['telecommuting', 'has_company_logo', 'has_questions']
target = 'fraudulent'

C:\Users\vardh\AppData\Local\Temp\ipykernel_17068\2366299236.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  job_data['description'].fillna('No information', inplace= True)
C:\Users\vardh\AppData\Local\Temp\ipykernel_17068\2366299236.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [11]:
# Preprocessing pipelines
text_pipeline = Pipeline([('tfidf', TfidfVectorizer(max_features= 5000))])
categorial_pipeline = Pipeline([('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer([('text', text_pipeline, 'description'), # Focus on 'description' for text modeling
                                  ('categorial', categorial_pipeline, 'location')], remainder='passthrough') # Include binary features as-is

# Prepare the data
X = job_data[text_features + categorial_features + binary_features]
y = job_data[target]

# Spliting The data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42, stratify= y)

X_train.shape, X_test.shape, y_train.value_counts(), y_test.value_counts()


((7026, 7),
 (1757, 7),
 fraudulent
 0    6668
 1     358
 Name: count, dtype: int64,
 fraudulent
 0    1667
 1      90
 Name: count, dtype: int64)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights to address imbalance
class_weights = compute_class_weight(
    class_weight= 'balanced',
    classes= np.array([0, 1]),
    y = y_train
)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

# Build and training the model PIPELINE
model_pipeline = Pipeline([('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier(class_weight= class_weight_dict, random_state= 42, n_estimators= 100, max_depth= None, min_samples_split= 2, min_samples_leaf= 1, max_features= 'sqrt'))])

# Training the Model
model_pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = model_pipeline.predict(X_test)

# Evaluate the MDOEL
report = classification_report(y_test, y_pred, target_names=['Non-Fraudulent', 'Fraudulent'])
conf_matrix = confusion_matrix(y_test, y_pred)

report, conf_matrix

ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

In [13]:
cv_scores = cross_val_score(model_pipeline, X_train, y_train, cv= 5, scoring= 'f1')
print(f"Cross-validation F1 scores: {cv_scores}")
print(f"Average F1 score: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\vardh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\vardh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\vardh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0